In [ ]:
# Verificar que el dataset se puede cargar correctamente
from ultralytics.data.utils import check_det_dataset
import yaml

# Cargar y verificar el archivo de configuración
with open("larc-2/data_fixed.yaml", "r") as f:
    data_config = yaml.safe_load(f)
    
print("Configuración del dataset:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

# Verificar que el dataset se puede cargar
try:
    dataset_info = check_det_dataset("larc-2/data_fixed.yaml")
    print("\n✅ Dataset verificado exitosamente!")
    print(f"Imágenes de entrenamiento: {len(dataset_info.get('train', []))}")
except Exception as e:
    print(f"\n❌ Error verificando dataset: {e}")

In [ ]:
import torch
import cv2
from roboflow import Roboflow
import dotenv
import os
dotenv.load_dotenv()
print("GPU disponible:", torch.cuda.is_available())

!nvidia-smi

In [ ]:
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace(os.getenv("ROBOFLOW_WORKSPACE")).project(os.getenv("ROBOFLOW_PROJECT_ID"))
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8m model
model = YOLO("yolov8m.pt")

# Train the model with the downloaded dataset
results = model.train(
    data="larc-2/data_fixed.yaml",  # Usar el archivo data.yaml corregido
    epochs=50,
    imgsz=640,
    batch=4,
    split=0.8,  # Dividir automáticamente: 80% entrenamiento, 20% validación
    perspective=0.0005,
    scale=0.4,
    translate=0.04,
    degrees=5,
    shear=1,
    hsv_s=0.5,
    hsv_v=0.3,
    flipud=0.2,
    patience=15,
    save_period=20,
    name="larc-model",
    project="runs/detect",
)